In [ ]:
!pip install opencv-python

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile aerial_data.yaml
train: /content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/images
val: /content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/images
nc: 1
names: ['car']

Overwriting aerial_data.yaml


In [ ]:
!cat aerial_data.yaml

train: /content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/images
val: /content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/images
nc: 1
names: ['car']


In [ ]:
import os

labels_dir = "/content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/labels"
bad_files = []

for filename in os.listdir(labels_dir):
    if filename.lower().endswith('.txt'):
        file_path = os.path.join(labels_dir, filename)
        with open(file_path, "r") as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] != "0":
                bad_files.append((filename, line.strip()))

if bad_files:
    print("Found labels not remapped to 0:")
    for fname, line in bad_files:
        print(f"{fname}: {line}")
else:
    print("All label files have class 0.")

All label files have class 0.


## Training The Model
using aerial_data.yaml which contains the path for training and label files


In [ ]:
%cd "/content/drive/MyDrive/DSCI_511_Project/yolov5_new"
!python train.py --img 640 --batch 16 --epochs 50 --data "/content/aerial_data.yaml" --weights yolov5s.pt

/content/drive/.shortcut-targets-by-id/1AS-UN1w4o3QpNZIHTPMkAHdSEMNGRZzn/DSCI_511_Project/yolov5_new
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-13 23:58:59.028861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741910339.051362    3778 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741910339.057925    3778 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B a

## Prediction using the trained model

In [ ]:
import os
import json
import torch
import random
import cv2

# Load your custom YOLOv5 model from exp14's best.pt checkpoint
model = torch.hub.load('ultralytics/yolov5', 'custom',
                       path='/content/drive/MyDrive/DSCI_511_Project/yolov5_new/runs/train/exp14/weights/best.pt',
                       force_reload=True)
model.conf = 0.25  # Adjust confidence threshold if needed

# Define the folder containing images for inference (adjust path as needed)
image_folder = "/content/drive/MyDrive/image"

# Folder to store rendered images with detections
output_folder = "/content/drive/MyDrive/DSCI_511_Project/results/validation"
os.makedirs(output_folder, exist_ok=True)

results_list = []       # List of dicts for JSON output
detection_results = {}  # To store full result objects for visualization and saving

# Loop over images in the folder and run inference
for filename in os.listdir(image_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder, filename)
        try:
            results = model(image_path)
        except FileNotFoundError:
            print(f"File not found, skipping: {filename}")
            continue
        detections = results.xyxy[0]  # [x1, y1, x2, y2, conf, class]
        car_count = detections.shape[0]  # each row corresponds to a detection
        results_list.append({
            "store_code": filename,
            "no of cars": int(car_count)
        })
        detection_results[filename] = results

        # Render the image with bounding boxes and save it
        rendered_imgs = results.render()  # returns list of rendered images
        for i, img in enumerate(rendered_imgs):
            # Construct output filename
            save_filename = f"validation_{filename}" if len(rendered_imgs)==1 else f"validation_{i}_{filename}"
            save_path = os.path.join(output_folder, save_filename)
            cv2.imwrite(save_path, img)

# Define output JSON path and save the results
json_path = "/content/drive/MyDrive/DSCI_511_Project/results/car_counts.json"
with open(json_path, "w") as f:
    json.dump(results_list, f, indent=4)

print("Results JSON saved to:", json_path)
print("Rendered images saved to:", output_folder)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-3-14 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultra

Results JSON saved to: /content/drive/MyDrive/DSCI_511_Project/results/car_counts.json
Rendered images saved to: /content/drive/MyDrive/DSCI_511_Project/results/validation


## Creating the Final JSON

In [3]:
import json

# Helper function to load a JSON file.
# It first attempts to load the file as a JSON array.
# If that fails (e.g. because the file is in NDJSON format), it loads line‑by‑line.
def load_json_file(filepath):
    try:
        with open(filepath, "r") as f:
            data = json.load(f)
        return data
    except json.JSONDecodeError:
        data = []
        with open(filepath, "r") as f:
            for line in f:
                line = line.strip()
                if line:
                    try:
                        data.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Error decoding a line in {filepath}: {e}")
        return data

# Define file paths
records_path = "/content/drive/MyDrive/records.json"
car_results_path = "/content/drive/MyDrive/DSCI_511_Project/results/car_counts.json"
final_results_path = "/content/drive/MyDrive/DSCI_511_Project/results/final_results.json"

# Load records.json
records_data = load_json_file(records_path)
print(f"Loaded {len(records_data)} records from {records_path}")

# Load car_counts.json
car_results_data = load_json_file(car_results_path)
print(f"Loaded {len(car_results_data)} entries from {car_results_path}")

# Remove duplicate place_ids from records.json (keeping the first occurrence)
unique_records = {}
for record in records_data:
    place_id = record["place_id"]
    if place_id not in unique_records:
        unique_records[place_id] = record
records_data_unique = list(unique_records.values())
print(f"After deduplication, {len(records_data_unique)} unique records remain.")

# Build a mapping from car_counts.json: strip ".png" from store_code and use the correct key "no of cars"
car_count_map = {}
for car in car_results_data:
    store_code = car["store_code"]
    if store_code.endswith(".png"):
        store_code = store_code[:-4]  # remove ".png"
    car_count_map[store_code] = car.get("no of cars", 0)

records_ids = set(unique_records.keys())
car_ids = set(car_count_map.keys())
print(f"Unique place_ids in records: {len(records_ids)}")
print(f"Unique store_codes in car_counts (after stripping .png): {len(car_ids)}")

mismatches_in_records = records_ids - car_ids
mismatches_in_car_results = car_ids - records_ids

if mismatches_in_records or mismatches_in_car_results:
    print("Mismatches found!")
    if mismatches_in_records:
        print("place_ids in records but not in car_counts:")
        print(mismatches_in_records)
    if mismatches_in_car_results:
        print("store_codes in car_counts but not in records:")
        print(mismatches_in_car_results)
else:
    print("All place_ids and store_codes match. Proceeding with merging...")

# Merge car counts into records
final_results = []
for record in records_data_unique:
    place_id = record["place_id"]
    # Set the no_of_cars value from car_count_map (will be non-zero if the mapping is correct)
    record["no_of_cars"] = car_count_map.get(place_id, 0)
    final_results.append(record)

# Save the merged results
with open(final_results_path, "w") as f:
    json.dump(final_results, f, indent=4)
print(f"Final results saved to {final_results_path}")

Loaded 1998 records from /content/drive/MyDrive/records.json
Loaded 1791 entries from /content/drive/MyDrive/DSCI_511_Project/results/car_counts.json
After deduplication, 1791 unique records remain.
Unique place_ids in records: 1791
Unique store_codes in car_counts (after stripping .png): 1791
All place_ids and store_codes match. Proceeding with merging...
Final results saved to /content/drive/MyDrive/DSCI_511_Project/results/final_results.json


In [5]:
import json
import pandas as pd

# Load the final results JSON
with open(final_results_path, "r") as f:
    final_results = json.load(f)

# Print the final JSON data (formatted)
print(json.dumps(final_results, indent=4))

# Convert the JSON data to a pandas DataFrame and print the first few rows
df = pd.DataFrame(final_results)
df.head(20)

[
    {
        "address": "Parking lot, Water St, Clinton, NJ 08809, United States",
        "location": {
            "lat": 40.6387329,
            "lng": -74.9117948
        },
        "name": "Municipal Lot",
        "place_id": "ChIJR8IPtAWLw4kROvacIJ6onxQ",
        "county": "Clinton County",
        "no_of_cars": 7
    },
    {
        "address": "NJ-31, Clinton, NJ 08809, United States",
        "location": {
            "lat": 40.6392178,
            "lng": -74.8938984
        },
        "name": "Clinton Park & Ride",
        "place_id": "ChIJR8t_I3eLw4kRmJNbQ9Z1a4Q",
        "county": "Clinton County",
        "no_of_cars": 15
    },
    {
        "address": "234 Main St, Hurleyville, NY 12747, United States",
        "location": {
            "lat": 41.7371061,
            "lng": -74.6734227
        },
        "name": "Morgan Outdoors",
        "place_id": "ChIJyZKFEAiZ3IkR0Y2NXWcrohQ",
        "county": "Sullivan County",
        "no_of_cars": 3
    },
    {
        "addre

,address,location,name,place_id,county,no_of_cars
0,"Parking lot, Water St, Clinton, NJ 08809, Unit...","{'lat': 40.6387329, 'lng': -74.9117948}",Municipal Lot,ChIJR8IPtAWLw4kROvacIJ6onxQ,Clinton County,7
1,"NJ-31, Clinton, NJ 08809, United States","{'lat': 40.6392178, 'lng': -74.8938984}",Clinton Park & Ride,ChIJR8t_I3eLw4kRmJNbQ9Z1a4Q,Clinton County,15
2,"234 Main St, Hurleyville, NY 12747, United States","{'lat': 41.7371061, 'lng': -74.6734227}",Morgan Outdoors,ChIJyZKFEAiZ3IkR0Y2NXWcrohQ,Sullivan County,3
3,"325 W Freedom Ave, Burnham, PA 17009, United S...","{'lat': 40.6304321, 'lng': -77.573883}",Mifflin County Commons,ChIJ18GSIGK1zokROnth9b2MgbY,Mifflin County,54
4,"842 W 4th St, Lewistown, PA 17044, United States","{'lat': 40.591275, 'lng': -77.5948901}",Dan Pierce Outdoor Shop,ChIJ53wMosJKyYkRSUmITaXRot4,Mifflin County,19
5,"7345 PA-655, Reedsville, PA 17084, United States","{'lat': 40.6587312, 'lng': -77.622937}",Hostetler Country Market,ChIJQ-X_VUS1zokRenmiLUAVIH8,Mifflin County,3
6,"34 Morgan Dr, Mcveytown, PA 17051, United States","{'lat': 40.4778482, 'lng': -77.77877459999999}",Nolt's Farm & Garden Inc,ChIJ_ePF83BQyYkRFrYOb2XmxT8,Mifflin County,3
7,"64 Forest Edge Ln, Lewistown, PA 17044, United...","{'lat': 40.6364735, 'lng': -77.4779076}",Carolus Trapping Supply,ChIJYT5tIK_LzokRDBwZ5PP8S5Q,Mifflin County,0
8,"5867 US-522, McClure, PA 17841, United States","{'lat': 40.6800825, 'lng': -77.4116385}",Point Store,ChIJ14epjXrOzokRArbqb-AIJ7w,Mifflin County,7
9,"150 Helen St, Lewistown, PA 17044, United States","{'lat': 40.5883966, 'lng': -77.5798216}",Lewistown Station,ChIJkY-F5ZVKyYkRCwmoQR7oFN8,Mifflin County,9
